# Parameters

In [ ]:
dest_dir = "/tmp/ghe_20210701"

# Read data and reference tables

In [ ]:
from owid import catalog
import json

In [ ]:
from etl.paths import REFERENCE_DATASET, BASE_DIR as base_path

In [ ]:
reference_dataset = catalog.Dataset(REFERENCE_DATASET)

In [ ]:
sex = reference_dataset["sex"]

In [ ]:
sex

In [ ]:
countries = reference_dataset["countries_regions"]

In [ ]:
countries.head()

In [ ]:
ghe_meadow = catalog.Dataset((base_path / "data/meadow/who/2021-07-01/ghe").as_posix())

In [ ]:
ghe_estimates = ghe_meadow["estimates"]

In [ ]:
ghe_estimates.shape

In [ ]:
ghe_estimates.head()

In [ ]:
country_codes = set(ghe_estimates.index.unique("country_code"))

In [ ]:
len(country_codes)

In [ ]:
owid_country_codes = set(countries.index)

In [ ]:
len(owid_country_codes)

In [ ]:
country_codes - owid_country_codes

💡 The set of (country codes in GHE) minus (owid table country codes) is empty, so we don't need to do any further harmonization here.

In [ ]:
ghe_estimates.index.unique("sex_code")

In [ ]:
with open(
    base_path
    / "etl"
    / "steps"
    / "data"
    / "garden"
    / "who"
    / "2021-07-01"
    / "ghe.mapping.sex.json"
) as f:
    ghe_to_owid_sex_codes = json.load(f)

In [ ]:
ghe_to_owid_sex_codes

In [ ]:
index_names = ghe_estimates.index.names

In [ ]:
ghe_estimates.reset_index(inplace=True)

In [ ]:
ghe_estimates["sex_code"] = ghe_estimates["sex_code"].cat.rename_categories(
    ghe_to_owid_sex_codes
)

In [ ]:
ghe_estimates.set_index(index_names, inplace=True)

In [ ]:
ghe_estimates.head()

In [ ]:
ghe_garden = catalog.Dataset.create_empty(dest_dir)

In [ ]:
ghe_garden.metadata = ghe_meadow.metadata
ghe_garden.save()

In [ ]:
ghe_garden.add(ghe_estimates)

In [ ]:
ghe_garden.save()